In [1]:
# Magic functions
%matplotlib
%load_ext autoreload
%autoreload 2



Using matplotlib backend: TkAgg


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
from calculate_zeta_t_lims import calculate_zeta_t_lims
from calculate_zeta_t_roots import calculate_zeta_t_roots
from find_marginalized_zeta_t_roots import find_marginalized_zeta_t_roots
import importlib
from log_C import log_C as log_C_func
import numpy as np
from plot_K_L import plot_K_L
import sys

In [42]:



## Constants
lambs = (0, 0.5, 1.0)
zeta_sp_abs_lim = 5.0
zeta_sp_steps = 100
dim = 1
n_pi = 4
ns = [100, 30, 10]
B = 10 # at least strong evidence against H0

# Relative prior uncertainties
# vs = [1, 2] 
vs = [1, 5, 10] 


log10_B = np.log10(B)

# Estimate the extent of Bayes factors for the given n, n_pi. Using logs base 10
ns = np.transpose(np.asarray(ns))
etas = np.sqrt(n_pi / (ns + n_pi))
if dim == 1:
    low_lims = np.log10(etas)
    pows = -ns - n_pi + 4
    upper_lims = np.multiply(pows, np.log10(etas))
elif dim == 2:
    low_lims = 2 * np.log10(etas)
    pows = 8 - 2 * (ns + n_pi)
    upper_lims = np.multiply(pows, np.log10(etas))
    
log10_B_lims = np.stack((low_lims, upper_lims), axis = 1)
# print(log10_B_lims)
# print(10**log10_B_lims)


# Create zeta_sp mesh
zeta_sps = np.linspace(- zeta_sp_abs_lim, zeta_sp_abs_lim, zeta_sp_steps)
    
# Calculate roots
lambs_count = len(lambs)
ns_count = len(ns)
vs_count = len(vs)
zeta_t_roots_size = (vs_count, ns_count, lambs_count + 1, 2, zeta_sp_steps)
zeta_t_roots = np.zeros(zeta_t_roots_size, dtype = np.float)


for v_ind in range(vs_count):
    v = vs[v_ind]
    
    for n_ind in range(ns_count):
        n = ns[n_ind]
        print("Processing v: %i/%i, n: %i/%i" % (v_ind + 1, vs_count, n_ind + 1, ns_count))

        # Check if the requested Bayes facto can be reached
        bl_reachable = log10_B >= log10_B_lims[n_ind, 0] and log10_B <= log10_B_lims[n_ind, 1]

        if not bl_reachable:
            raise ValueError("The value of the Bayes factor (B = %.3g) cannot be achieved for the current parameters. Bayes factor limits: [%.3g; %.3g]" 
                             % (10 ** log10_B, 10 ** log10_B_lims[n_ind, 0], 10 ** log10_B_lims[n_ind, 1]))

        for zeta_ind in range(zeta_sp_steps):
            zeta_sp = zeta_sps[zeta_ind]

            # Fixed-lambda models
            for lamb_ind in range(lambs_count):
                lamb = lambs[lamb_ind]

                cur_roots = calculate_zeta_t_roots(zeta_sp = zeta_sp, lamb = lamb, n = n, n_pi = n_pi, B = B, v = v,
                                                   dim = dim)
                # Save
                zeta_t_roots[v_ind, n_ind, lamb_ind, 0, zeta_ind] = cur_roots[0]
                zeta_t_roots[v_ind, n_ind, lamb_ind, 1, zeta_ind] = cur_roots[1] 

            # Marginalized
            cur_roots = find_marginalized_zeta_t_roots([zeta_sp], n, n_pi, B, v, dim = dim)
            # Save
            zeta_t_roots[v_ind, n_ind, lambs_count, 0, zeta_ind] = cur_roots[0]
            zeta_t_roots[v_ind, n_ind, lambs_count, 1, zeta_ind] = cur_roots[1]

print("Calculation terminated")
        
# print(zeta_t_roots)



Processing v: 1/3, n: 1/3
Processing v: 1/3, n: 2/3
Processing v: 1/3, n: 3/3
Processing v: 2/3, n: 1/3
Processing v: 2/3, n: 2/3
Processing v: 2/3, n: 3/3
Processing v: 3/3, n: 1/3
Processing v: 3/3, n: 2/3
Processing v: 3/3, n: 3/3
Calculation terminated


In [43]:
# Plot
plot_K_L(zeta_sps, zeta_t_roots, ns, vs, dim = dim)

In [ ]:
from find_ito_roots import find_ito_roots
find_ito_roots(n_0, n, n_pi)

In [ ]:
from calculate_marginalized_integral import calculate_marginalized_integral
calculate_marginalized_integral([0.5], [1], 10, 4)

In [ ]:
2D: [[ -1.41497335 141.4973348 ]
 [ -0.92941893  27.88256777]
 [ -0.54406804   5.44068044]]
    
1D: [[-0.70748667 70.7486674 ]
 [-0.46470946 13.94128389]
 [-0.27203402  2.72034022]]
    

In [ ]:
y = [-1, 1]

In [ ]:
print (K)

In [31]:
from find_marginalized_zeta_sp_limit import find_marginalized_zeta_sp_limit

Ks = np.exp([-3, 0, 3])
n_max = 200
ns = range(1, n_max + 1)

Ks_count = len(Ks)

zeta_sp_borders = np.zeros((Ks_count, n_max), dtype = np.float)

# Calculate roots for different n and K
for K_ind in range(Ks_count):
    K = Ks[K_ind]
    for n_ind in range(n_max):
        n = ns[n_ind]
        zeta_sp_borders[K_ind, n_ind] = find_marginalized_zeta_sp_limit(n = n, n_pi = 4, K = K)
# print(zeta_sp_borders)


In [32]:
from plot_zeta_sp_borders import plot_zeta_sp_borders
plot_zeta_sp_borders(ns, zeta_sp_borders)